In [1]:
import json

import random
from datetime import datetime, timedelta

def random_timestamp(start_date=datetime(2000, 1, 1, 0, 0, 0), end_date=datetime(2030, 12, 31, 23, 59, 59)):
    """Generate a random timestamp string between two dates"""
    # Convert to timestamps (seconds since epoch)
    start_timestamp = datetime.timestamp(start_date)
    end_timestamp = datetime.timestamp(end_date)
    
    # Generate a random timestamp within the range
    random_timestamp = random.uniform(start_timestamp, end_timestamp)
    
    # Convert back to datetime and format as string
    random_date = datetime.fromtimestamp(random_timestamp)
    return random_date.strftime("%Y-%m-%d %H:%M:%S")


source_files = ["gallery.json", "otel-demo.json", "instruction_benchmark_dev_set.json"]
examples = []
for file in source_files:
    content = json.load(open(file, "r"))
    print(f"Loaded {len(content)} examples from {file}")
    for sample in content:
        examples.append({
            "question": sample["question"],
            "query": sample["query"],
            "now": random_timestamp(),
        })
        
        
do_write = False
if do_write:
    with open("examples.json","w") as f:
        json.dump(examples, f, indent=4)

Loaded 11 examples from gallery.json
Loaded 9 examples from otel-demo.json
Loaded 34 examples from instruction_benchmark_dev_set.json


In [2]:
with open("examples.json") as f:
    examples = json.load(f)

In [3]:
examples[:3]

[{'question': 'What is the average response time for each API endpoint in our application over the last hour?',
  'query': 'source=api_logs | where timestamp > now() - 1h | stats avg(response_time) as avg_response_time by endpoint',
  'now': '2015-08-31 09:47:56'},
 {'question': 'What is the CPU, memory, and disk usage across our three main server clusters over the last 24 hours?',
  'query': 'source=server_metrics | where timestamp > now() - 24h | stats avg(cpu_usage) as avg_cpu, avg(memory_usage) as avg_memory, avg(disk_usage) as avg_disk by cluster',
  'now': '2012-05-07 05:32:24'},
 {'question': 'What is the distribution of different types of system errors across months?',
  'query': 'source = system_logs | stats count() by span(timestamp, 1M) as month, error_type',
  'now': '2024-07-29 20:14:04'}]

In [4]:
import os
import re
import boto3

from tqdm import tqdm

def setup_bedrock():
    session = boto3.Session(
        region_name=os.environ.get("AWS_REGION", "us-east-1"),
    )
    return session.client("bedrock-runtime")

bedrock_client = setup_bedrock()

def get_prompt(q, cur_time):
    return f"""
    Here is a user question. We'll transform the user question to a SQL query. Now you need to parse the time filter for sql query.
    You need to give the extracted query and time filter range. Here are several rules:
    1. Try infer the time filter range from user input. The time range should be calculated based on current time.
    2. important: always return null when user question doesn't specify a time filter range. Otherwise we got wrong query.
    3. Extracted query should just contain user intention and without time range information.
    4. The output format is XML. Use tag <query> <time_filter> <start> and <end>. Always print these info in order.
    5. Only give me the XML tags and content, no other words.
    6. Distinguish the time-related keyword (daily, per month, monthly, per week etc) is for aggregation or for time filter. They are different.

    <User input>
    <user question>{q}</user question>
    <current timestamp>{cur_time}</current timestamp>
    </User input>

    Here are some examples:

    <example input>
    <user question>show me the city has max revenue in last year</user question>
    <current timestamp>2025-05-01 21:12:44</current timestamp>
    </example input>
    <example output>
    <query>show me the city has max revenue</user question>
    <time_filter>last year</time_filter>
    <start>2024-01-01 00:00:00</start>
    <end>2025-01-01 00:00:00</end>
    </example output>


    <example input>
    <user question>total order number in last week</user question>
    <current timestamp>2025-01-04 06:01:54</current timestamp>
    </example input>
    <example output>
    <query>show me the total order number</user question>
    <time_filter>last week</time_filter>
    <start>2024-12-28 00:00:00</start>
    <end>2025-01-04 00:00:00</end>
    </example output>


    <example input>
    <user question>show me the monthly total revenue</user question>
    <current timestamp>2025-11-21 02:21:27</current timestamp>
    </example input>
    <example output>
    <query>show me the monthly total revenue</user question>
    <time_filter>no explicit time range info</time_filter>
    <start>null</start>
    <end>null</end>
    </example output>
    """

def parse_tagged_string(text):
    """
    解析格式为<tag>value</tag>的字符串为字典
    """
    # 正则表达式匹配<tag>value</tag>模式
    pattern = r'<(\w+)>(.*?)</\1>'
    
    # 查找所有匹配项
    matches = re.findall(pattern, text, re.DOTALL)
    
    # 转换为字典
    result = {}
    for tag, value in matches:
        # 处理"null"值
        if value.strip() == "null":
            result[tag] = None
        else:
            result[tag] = value.strip()
    
    return result

def invoke(q, cur_time):
    response = bedrock_client.invoke_model(
        modelId="anthropic.claude-3-haiku-20240307-v1:0",
        body=json.dumps(
            {
                "temperature": 0,
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 8192,
                "messages": [
                    {
                        "role": "user",
                        "content": [{"type": "text", "text": get_prompt(q, cur_time)}],
                    }
                ],
            }
        ),
    )
    
    result = json.loads(response.get("body").read())
    generated_text = result["content"][0]["text"]
    return parse_tagged_string(generated_text)

In [ ]:
predicts = []
for example in tqdm(examples):
    predicts.append(invoke(example["question"], example["now"]))

In [ ]:
for i in range(len(examples)):
    examples[i]["start"] = predicts[i]["start"]
    examples[i]["end"] = predicts[i]["end"]
    
do_write = False
if do_write:
    with open("time_bench.json","w") as f:
        json.dump(examples, f, indent=4)

## manually update the golden label